In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%debug

ERROR:root:No traceback has been produced, nothing to debug.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
sys.path.insert(0, "../../code-previous")

In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator

import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
from plotly import tools

import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.feature_extraction.text import CountVectorizer


from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/all_discipline_data.pql"
data50 = pd.read_pickle(data_path)

In [6]:
#data50 = data
#data  = None

In [7]:
data50 = data50[data50.PubYear == 1950]

data50.head()

,ArticleID,Type,Label,PubYear,Doctypes,Title,Journal,Journal Abbreviation ISO,title_without_stopwords
304370433,A1950UM47300017,heading,Science & Technology,1950,Note,CHAPPING OF THE SKIN ON RETURNING FROM THE TRO...,ARCHIVES OF DERMATOLOGY AND SYPHILOLOGY,,chapping skin returning from tropics cooler area
304370434,A1950UM47300017,subheading,Life Sciences & Biomedicine,1950,Note,CHAPPING OF THE SKIN ON RETURNING FROM THE TRO...,ARCHIVES OF DERMATOLOGY AND SYPHILOLOGY,,chapping skin returning from tropics cooler area
304370435,A1950UM47300017,ASCA traditional subject,Dermatology,1950,Note,CHAPPING OF THE SKIN ON RETURNING FROM THE TRO...,ARCHIVES OF DERMATOLOGY AND SYPHILOLOGY,,chapping skin returning from tropics cooler area
304370436,A1950UM47300017,ASCA extended subject,Dermatology,1950,Note,CHAPPING OF THE SKIN ON RETURNING FROM THE TRO...,ARCHIVES OF DERMATOLOGY AND SYPHILOLOGY,,chapping skin returning from tropics cooler area
304370437,A1950YF49700012,ASCA extended subject,General & Internal Medicine,1950,Article,*SERONEGATIVT GUMMA,NORDISK MEDICIN,,seronegativt gumma


In [8]:
data50 = data50.sort_values(by="PubYear")

In [19]:
data50.head()

,ArticleID,Type,Label,PubYear,Doctypes,Title,Journal,Journal Abbreviation ISO,title_without_stopwords
304370433,A1950UM47300017,heading,Science & Technology,1950,Note,CHAPPING OF THE SKIN ON RETURNING FROM THE TRO...,ARCHIVES OF DERMATOLOGY AND SYPHILOLOGY,,chapping skin returning from tropics cooler area
304558972,A1950YB35900013,ASCA extended subject,Astronomy & Astrophysics,1950,Article,"THE SPECTRUM OF R-AQUARII, 1936-1949",ASTROPHYSICAL JOURNAL,Astrophys. J.,spectrum r aquarii 1936 1949
304558973,A1950YB35900013,ASCA traditional subject,Astronomy & Astrophysics,1950,Article,"THE SPECTRUM OF R-AQUARII, 1936-1949",ASTROPHYSICAL JOURNAL,Astrophys. J.,spectrum r aquarii 1936 1949
304558974,A1950YB35900013,subheading,Physical Sciences,1950,Article,"THE SPECTRUM OF R-AQUARII, 1936-1949",ASTROPHYSICAL JOURNAL,Astrophys. J.,spectrum r aquarii 1936 1949
304558975,A1950YB35900013,heading,Science & Technology,1950,Article,"THE SPECTRUM OF R-AQUARII, 1936-1949",ASTROPHYSICAL JOURNAL,Astrophys. J.,spectrum r aquarii 1936 1949


In [9]:
data50_group = data50[["Label","title_without_stopwords"]].groupby(data50.Label)["title_without_stopwords"].apply(list)

data50_group = pd.DataFrame(data50_group)

In [10]:
data50_group["title_words"] = data50_group["title_without_stopwords"].progress_apply(lambda x: " ".join(x))

Progress: 100%|██████████| 613/613 [00:00<00:00, 25702.86it/s]


In [11]:
data50.head()

,ArticleID,Type,Label,PubYear,Doctypes,Title,Journal,Journal Abbreviation ISO,title_without_stopwords
304370433,A1950UM47300017,heading,Science & Technology,1950,Note,CHAPPING OF THE SKIN ON RETURNING FROM THE TRO...,ARCHIVES OF DERMATOLOGY AND SYPHILOLOGY,,chapping skin returning from tropics cooler area
304558972,A1950YB35900013,ASCA extended subject,Astronomy & Astrophysics,1950,Article,"THE SPECTRUM OF R-AQUARII, 1936-1949",ASTROPHYSICAL JOURNAL,Astrophys. J.,spectrum r aquarii 1936 1949
304558973,A1950YB35900013,ASCA traditional subject,Astronomy & Astrophysics,1950,Article,"THE SPECTRUM OF R-AQUARII, 1936-1949",ASTROPHYSICAL JOURNAL,Astrophys. J.,spectrum r aquarii 1936 1949
304558974,A1950YB35900013,subheading,Physical Sciences,1950,Article,"THE SPECTRUM OF R-AQUARII, 1936-1949",ASTROPHYSICAL JOURNAL,Astrophys. J.,spectrum r aquarii 1936 1949
304558975,A1950YB35900013,heading,Science & Technology,1950,Article,"THE SPECTRUM OF R-AQUARII, 1936-1949",ASTROPHYSICAL JOURNAL,Astrophys. J.,spectrum r aquarii 1936 1949


In [12]:
len(list(data50_group.index))

613

In [20]:
import pickle
filename = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/data1950_grouped_by_labels.pickle"
pickle_out = open(filename,"wb")
pickle.dump(data50_group, pickle_out)
pickle_out.close()


# with open(filename, 'rb') as handle:
#     data50 = pickle.load(handle)

In [21]:
data50_group.head()

,title_without_stopwords,title_words
Label,,
1ST YEAR,[hyperbrachycephaly influenced cultural condit...,hyperbrachycephaly influenced cultural conditi...
9-YEAR-OLD WHITE MALES,[distribution subcutaneous],distribution subcutaneous
ABILITIES,"[intellectual changes during maturity, applica...",intellectual changes during maturity applicati...
ACCEPTANCE,[counseling methods therapy],counseling methods therapy
ACID,[connective tissue fine structure some methods...,connective tissue fine structure some methods ...


In [22]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer

#vectorizer = CountVectorizer(min_df=2, max_df=1.0)

#vectorizer = CountVectorizer(min_df=4, max_df=1.0)
#vectorized_words = vectorizer.fit_transform(data50_group.title_words)

# saved with min_df=4, max_df=1.0

vectorizer = CountVectorizer(min_df=2, max_df=1.0)
vectorized_words = vectorizer.fit_transform(data50_group.title_words)

vectorizer_file = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/vectorized1950/vectorizer_1950.pickle"
pickle_out = open(vectorizer_file,"wb")
pickle.dump(vectorizer, pickle_out)
pickle_out.close()

vectorized_words_file = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/vectorized1950/vectorized_words_1950.pickle"

pickle_out = open(vectorized_words_file,"wb")
pickle.dump(vectorized_words_file, pickle_out)
pickle_out.close()



# or read from pickle

# with open(vectorizer_file, 'rb') as handle:
#     vectorizer = pickle.load(handle)

# with open(vectorized_words_file, 'rb') as handle:
#     vectorized_words = pickle.load(handle)    

In [23]:
#1. Document Frequency - Number of disciplines in which the word has appeared

vectorized_wordarray = vectorized_words.toarray()
document_frequency = (vectorized_wordarray != 0).sum(0).tolist()
document_frequency_log =  [np.log(x) for x in document_frequency]
#2. Term Frequency - Number of times in which a word appears in a document
term_frequency = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
term_frequency_log = [np.log(x) for x in term_frequency]

In [17]:
# import pickle
# pickle_out = open("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/vectorizer_words_whole_corpus.pickle","wb")
# pickle.dump(vectorizer, pickle_out)
# pickle_out.close()

In [ ]:
# import pickle
# pickle_out = open("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/vectorized_words_whole_corpus.pickle","wb")
# pickle.dump(vectorized_words, pickle_out)
# pickle_out.close()

In [24]:
count_vocab = {value:key for key,value in vectorizer.vocabulary_.items()}
count_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
words = [count_rev_dict[i] for i in range(len(vectorizer.vocabulary_.keys()))]

In [41]:
#tf_idf_vocab = {value:key for key,value in vectorizer.vocabulary_}
#tf_idf_vocab
#tfidf_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
#words = [tfidf_rev_dict[i] for i in range(len(vectorizer.idf_))]

In [25]:
# consider one discipline - Applied Physics index 449

# get tf scores of all the words of applied physics
#tf_applied_physics = tf_idf_matrix[450:451,:].todense().reshape(-1,).tolist()[0]
#idf_words = vectorizer.idf_

#tf_idf_physics = [(tf_applied_physics[i], idf_words[i]) for i in range(len(idf_words))]

trace0 = go.Scatter(
    x = document_frequency_log,
    y = term_frequency_log,
    mode = 'markers',
    text = words,
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)


layout= go.Layout(
    title= 'Document Frequency - Term Frequency plot ',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Log Document Frequency (Number of disciplines in which the word occurs)',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Log Term Frequency (Count of the word in corpus)',
        ticklen= 5,
        gridwidth= 2,
    ),
    
    showlegend= False
)

plotdata = [trace0]


fig= go.Figure(data=plotdata, layout=layout)
py.iplot(fig)

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:


The draw time for this plot will be slow for clients without much RAM.


/usr/local/lib/python3.6/dist-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



Let us pick a couple of "good" words (words which occur in multiple disciplines and are whose log tf are not too high) for analysis

__Words:  neutron, influenza, ketones, protons, tuberculin__

In [69]:
# which all disciples have neutron in them ?
# lets find the indices


# index of the word - 
index = 37037
words.index("neutron")
count_rev_dict[index]
counts = vectorized_wordarray[:,index]

discipline_indices = [discipline_index for discipline_index in range(len(counts.tolist())) if counts[discipline_index] != 0]

9

__Which disciplines are those?__

In [73]:
[list(data50_group.index)[index] for index in discipline_indices]

['Chemistry',
 'Chemistry, Multidisciplinary',
 'Chemistry, Physical',
 'Instruments & Instrumentation',
 'Life Sciences & Biomedicine',
 'Multidisciplinary Sciences',
 'Physical Sciences',
 'Physics',
 'Physics, Applied',
 'Physics, Atomic, Molecular & Chemical',
 'Physics, Multidisciplinary',
 'Physics, Nuclear',
 'Radiology, Nuclear Medicine & Medical Imaging',
 'Science & Technology',
 'Science & Technology - Other Topics',
 'Technology']